# Integration on level set domains
* simple example: calculate area of the unit circle using level set representations
* background domain: $[-1.5, 1.5]^2 \subset \mathbb{R}^2$.

Let $\phi$ be a continuous level set function. We recall:
$$
  \Omega_{-} := \{ \phi < 0 \}, \quad
  \Omega_{+} := \{ \phi > 0 \}, \quad
  \Gamma := \{ \phi = 0 \}.
$$

In [1]:
#import netgen.gui

from math import pi
from ngsolve import *
# basic xfem functionality
from xfem import *
# for isoparametric mapping
from xfem.lsetcurv import *

from ngsolve.webgui import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import *

We generate the background mesh of the domain and use a simplicial triangulation

In [2]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.8, quad_dominated=False))
Draw(mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-109-gfd7edeb77', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1…

On the background mesh we define the level set function. In this example we choose a levelset function which measures the distance to the origin and substracts a constant $r$, resulting in an circle of radius $r$ as $\Omega_{-}$:

In [3]:
r=1
levelset = sqrt(x**2+y**2)-r
Draw(levelset, mesh, "Levelset",min=0,max=0)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-109-gfd7edeb77', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

* Area of the unit circle:
$$
A = \pi = \int_{\Omega_-(\phi)} 1 \, \mathrm{d}x
$$

* The ngsxfem integration on implicitly defined geometries is only able to handle functions $\phi$ which are in $\mathcal{P}^1(T)$ for all $T$ of the triangulation.

* Therefore we replace the levelset function $\phi$ with an approximation $\phi^\ast \in \mathcal{P}^1(T), T \subset \Omega$. 

In [4]:
V = H1(mesh,order=1)
lset_approx = GridFunction(V)
InterpolateToP1(levelset,lset_approx)
Draw(lset_approx,mesh,"lsetp1",min=0,max=0.0)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-109-gfd7edeb77', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

Notice that this replacement $\phi \mapsto \phi^\ast$ will introduce a geometry error of order $h^2$. We will investigate this numerically later on in this notebook.

As a next step we need to define the integrand:

In [5]:
f = CoefficientFunction(1)

To call the `Integrate` function -- additionally to the arguments known from standard NGSolve -- we need to pass the "levelset_domain" dictionary argument which has the following entries:
* "levelset": level set function which describes the geometry (best choice: P1-GridFunction)
* "domain_type": decision on which part of the geometry the integration should take place (`NEG`/`POS`/`IF`)

In [6]:
order = 2
integral = Integrate(levelset_domain = { "levelset" : lset_approx, "domain_type" : NEG},
                             cf=f, mesh=mesh, order = order)
error = abs(integral - pi)
print("Result of the integration: ", integral)
print("Error of the integration: ", error)

Result of the integration:  2.897369547779562
Error of the integration:  0.2442231058102311


### Convergence 

To get more accurate results
* save the previous error and execute a refinement 
* repeat the steps from above
* Repeadetly execute the lower block to add more refinement steps.

In [7]:
errors = []

def AppendResultAndPostProcess(integral):
    error = abs(integral - pi)
    print("Result of the integration: ", integral)
    print("Error of the integration: ", error)

    errors.append(error)
    eoc = [log(errors[i+1]/errors[i])/log(0.5) for i in range(len(errors)-1)]

    print("Collected errors:", errors)
    print("experimental order of convergence (L2):", eoc)    

AppendResultAndPostProcess(integral)

Result of the integration:  2.897369547779562
Error of the integration:  0.2442231058102311
Collected errors: [0.2442231058102311]
experimental order of convergence (L2): []


In [8]:
# refine cut elements only:
RefineAtLevelSet(gf=lset_approx)
mesh.Refine()

V.Update()
lset_approx.Update()

InterpolateToP1(levelset,lset_approx)
Draw(lset_approx,mesh,"lsetp1",min=0,max=0.0)

integral = Integrate(levelset_domain = { "levelset" : lset_approx, "domain_type" : NEG},
                     cf = f, mesh = mesh, order = order)
AppendResultAndPostProcess(integral)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-109-gfd7edeb77', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

Result of the integration:  3.064299809805666
Error of the integration:  0.07729284378412693
Collected errors: [0.2442231058102311, 0.07729284378412693]
experimental order of convergence (L2): [1.6597929469020138]


We observe only a second order convergence which result from the geometry approximation.

## Higher order geometry approximation with an isoparametric mapping
In order to get higher order convergence we can use the isoparametric mapping functionality of xfem.

We apply a mesh transformation technique in the spirit of isoparametric finite elements:
![title](graphics/lsetcurv.jpg)

### Video of the mesh transformation

In [9]:
%%html
<iframe width=100% height=600px src="https://www.youtube.com/embed/Mst_LvfgPCg?rel=0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

To compute the corresponding mapping we use the LevelSetMeshAdaptation class

In [10]:
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2, threshold=1000, discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
Draw(deformation,mesh,"deformation")

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-109-gfd7edeb77', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

We can observe the geometrical improvement in the following sequence:

In [11]:
Draw(lsetmeshadap.lset_p1,mesh,"lsetp1")
Draw(deformation,mesh,"deformation")

visoptions.autoscale = 0
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 1.0
visoptions.subdivisions = 4
from time import sleep

N=10000
#from time import sleep
deformation.vec.data = 1.0/N*deformation.vec
for i in range (1,N+1):
    #sleep(0.1)
    deformation.vec.data = (i+1)/i * deformation.vec
    Redraw()

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-109-gfd7edeb77', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-109-gfd7edeb77', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

## Convergence study with isoparametric mapping applied
### error tables:

In [12]:
from xfem.lsetcurv import *

order = 2
refinements = 7
mesh = Mesh (square.GenerateMesh(maxh=0.8, quad_dominated=False))

levelset = sqrt(x*x+y*y)-1
referencevals = { POS : 9-pi, NEG : pi, IF : 2*pi }

lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.2, discontinuous_qn=True)
lsetp1 = lsetmeshadap.lset_p1
errors_uncurved = dict()
errors_curved = dict()
eoc_uncurved = dict()
eoc_curved = dict()

for key in [NEG,POS,IF]:
    errors_curved[key] = []
    errors_uncurved[key] = []
    eoc_curved[key] = []
    eoc_uncurved[key] = []

### refinements:

In [13]:

f = CoefficientFunction (1.0)

for reflevel in range(refinements):
    if(reflevel > 0):
        mesh.Refine()

    for key in [NEG,POS,IF]:
        # Applying the mesh deformation
        deformation = lsetmeshadap.CalcDeformation(levelset)

        integrals_uncurved = Integrate(levelset_domain = { "levelset" : lsetp1, "domain_type" : key},
                                        cf=f, mesh=mesh, order = order)

        mesh.SetDeformation(deformation)
        integrals_curved = Integrate(levelset_domain = { "levelset" : lsetp1, "domain_type" : key},
                                       cf=f, mesh=mesh, order = order)
        # Unapply the mesh deformation (for refinement)
        mesh.UnsetDeformation()

        errors_curved[key].append(abs(integrals_curved - referencevals[key]))
        errors_uncurved[key].append(abs(integrals_uncurved - referencevals[key]))
    # refine cut elements:
    RefineAtLevelSet(gf=lsetmeshadap.lset_p1)



### Convergence rates:

In [14]:
for key in [NEG,POS,IF]:
    eoc_curved[key] = [log(a/b)/log(2) for (a,b) in zip (errors_curved[key][0:-1],errors_curved[key][1:]) ]
    eoc_uncurved[key] = [log(a/b)/log(2) for (a,b) in zip (errors_uncurved[key][0:-1],errors_uncurved[key][1:]) ]

print("errors (  curved):  \n{}\n".format(  errors_curved))
print("   eoc (  curved):  \n{}\n".format(     eoc_curved))

errors (  curved):  
{<DOMAIN_TYPE.NEG: 0>: [0.018414951196107587, 0.002576506961442515, 5.068159427690233e-06, 1.2107751648837706e-05, 1.001608783468555e-06, 5.652403123335148e-08, 2.908470353446546e-09], <DOMAIN_TYPE.POS: 1>: [0.15822943946394652, 0.002576506961441183, 5.068159428134322e-06, 1.2107751654610865e-05, 1.001608787909447e-06, 5.6524040559224886e-08, 2.9084805674983727e-09], <DOMAIN_TYPE.IF: 2>: [0.023146829325124685, 0.0022224244347226474, 1.1003235001361134e-05, 1.3157275289721326e-05, 1.0638499459147965e-06, 6.064207092038032e-08, 3.177648366659014e-09]}

   eoc (  curved):  
{<DOMAIN_TYPE.NEG: 0>: [2.8373891781517377, 8.98973886751225, -1.2563971756187544, 3.595539964318788, 4.147310947920558, 4.280531857098421], <DOMAIN_TYPE.POS: 1>: [5.9404577454642915, 8.98973886738509, -1.2563971761802395, 3.5955399586101247, 4.147310716287562, 4.280527028637952], <DOMAIN_TYPE.IF: 2>: [3.3806083137471026, 7.658062811136652, -0.2579330105145717, 3.6284941747526154, 4.132831843236782

* The integration on level set domains (and the mapping) can be extended to `SymbolicBFI` / `SymbolicLFI` ...